In [1]:
from scipy.io import loadmat
from matplotlib import pyplot as plt
import os.path
import os
import numpy as np

from image_training import create_model, create_generators, train_model

Using TensorFlow backend.


In [2]:
def zero_padding(images, max_frames):
    new_ims = []
    for im in images:
        num_frames = im['X'].shape[2]
        if num_frames < max_frames:
            num_pad = max_frames-num_frames
            im_new = np.pad(im['X'], ((0, 0), (0, 0), (0, num_pad)), 'constant', constant_values = 0)
        new_ims.append(im_new)
    return new_ims

def load_images(folder):
    images, classification = [], [] 
    for i in range(1, 202):
        data_file = np.load('data/resized/fractals_' + str(i) + '.npy').item()
        classif = data_file['TICI_report'][0]
        if str(classif) == '[nan]':
            continue
        elif classif == '2a' or classif == '2b':
            images.append(data_file['X'])
            classification.append(classif)
        else:
            images.append(data_file['X'])
            classification.append(classif[0])
            
    return images, classification

In [6]:
# crossvalidation 
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from scipy.io import loadmat
import os

cv = StratifiedKFold(5)

tprs = []
aucs = []
i = 1

#not sure if there's a different folder but this can't be left blank like it was before and not sure if there is a preprocessed data set I should be pulling from

folder = 'data/resized/'

images, classification = load_images(folder)

images = np.array(images)
imlist_shape = images.shape
images = images.reshape(imlist_shape[0], imlist_shape[1], imlist_shape[2], imlist_shape[3], 1)

model = create_model(max_frames=39)
batch_size = 32

for train, test in cv.split(images, classification):
    x_train = images[train]
    y_train = classification[train]
    x_test = images[test]
    y_test = classification[test]
    
    generators = create_generators(x_train, y_train, batch_size)
    model = train_model(model, 10, generators, len(x_train), batch_size)
    
    Y_pred = model.predict(x_test)
    fpr, tprs, thresholds = roc_curve(y_test, Y_pred)
    tpr.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw = 1, alpha = 0.3, label = 'ROC fold %d (AUC - %0.2f)' % (i, roc_auc))
    plt.legend()
    i+=1
    
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Random Chance', alpha=.8)

mean_fpr = np.linspace(0, 1, 100)
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = metrics.auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc))

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('PLS-DA ROC Curve with Crossvalidation')
plt.legend(loc="lower right")
plt.savefig('ROC Curve for PLS-DA with Crossvalidation-14comp.jpg')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:18: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


ValueError: Input 0 is incompatible with layer conv3d_1: expected ndim=5, found ndim=4